In [1]:
import numpy as np
A = np.arange(4).reshape((2, 2))
d = np.array((1, 2))
D = np.diag(d)
B = D.dot(A)
d2 = np.expand_dims(d, -1)
C = d2 * A
E = np.dot(A, d)

# now try multiplying the same rank 2 matrix by a stack of vectors
I = np.eye(2)
I[1,1] += 1
B = np.arange(20).reshape((-1, 2))
G = np.einsum('jk,ik->ji', B, I)

# now try multiplying a stack of matrices by a stack of matrices
F = np.tile(I, (5,1,1))
for i in range(5):
    F[i,1,0] += i
H = np.einsum('ijk,ilk->ijl', F, F)

# now try multiplying a stack of matrices by a stack of diagonal (vectors, but interpreted as matrices)
D2 = np.tile(np.array((2, 1)), (5, 1))
U = np.einsum('ijk,ik->ijk', F, D2)

# now try multiplying by the diagonal
ones = np.ones(4).reshape((2, 2))
L = np.einsum('ij,j->ij', ones, d)
print(L)

[[1. 2.]
 [1. 2.]]


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import tensorflow as tf
import numpy as np
A = tf.constant(np.arange(8).reshape((2, 2, 2)))
d = tf.constant(np.arange(4).reshape((2, 2)) + 1)
D = tf.linalg.diag(d)
B = tf.matmul(D, A)
d2 = tf.expand_dims(d, -1)
C = d2 * A
E = tf.einsum('ijk,ik->ij', A, d)

# test making a stacked identity
I_ = tf.expand_dims(tf.eye(3), 0)
I = tf.tile(I_, (5, 1, 1))

# testing matrix_band_part
G = tf.constant(np.random.normal(size=(5, 3, 3)))
G_ld = tf.matrix_band_part(G, -1, 0) - tf.matrix_band_part(G, 0, 0)
G_ud = tf.einsum('ijk->ikj', G_ld)

with tf.Session() as sess:
    items = sess.run([G_ld, G_ud])
    
for i in items:
    print(i)
    print(i.shape)
    print('-' * 15)

[[[ 0.          0.          0.        ]
  [-0.33040237  0.          0.        ]
  [ 1.2953239   1.77695692  0.        ]]

 [[ 0.          0.          0.        ]
  [ 0.49417969  0.          0.        ]
  [ 2.45265689 -0.44301211  0.        ]]

 [[ 0.          0.          0.        ]
  [-1.25009695  0.          0.        ]
  [ 0.2038571   0.93812283  0.        ]]

 [[ 0.          0.          0.        ]
  [ 0.31359143  0.          0.        ]
  [-0.53848896 -0.86957616  0.        ]]

 [[ 0.          0.          0.        ]
  [ 0.62118275  0.          0.        ]
  [-0.83217765  1.70415169  0.        ]]]
(5, 3, 3)
---------------
[[[ 0.         -0.33040237  1.2953239 ]
  [ 0.          0.          1.77695692]
  [ 0.          0.          0.        ]]

 [[ 0.          0.49417969  2.45265689]
  [ 0.          0.         -0.44301211]
  [ 0.          0.          0.        ]]

 [[ 0.         -1.25009695  0.2038571 ]
  [ 0.          0.          0.93812283]
  [ 0.          0.          0.        ]]